In [ ]:
import os

from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "fridon-ai"

os.environ["PLAYER_TEMPLATE_PATH"] = "./data/templates/"

# from app.core.graph.workflows import graph

In [ ]:
from langchain_openai import ChatOpenAI
from app.core.graph import create_graph
from app.core.plugins.registry import ensure_plugin_registry


plugin_names = ["greeter", "kamino", "jupyter", "symmetry", "coin-price-chart-similarity-search", "coin-technical-analyzer"]
registry = ensure_plugin_registry()

plugins = [registry.plugins[plugin_name]() for plugin_name in plugin_names]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

graph = create_graph(llm, plugins, memory=AsyncSqliteSaver.from_conn_string(":memory:"))


In [ ]:
from IPython.display import display, HTML
import base64


def display_image(image_bytes: bytes, width=1500):
    decoded_img_bytes = base64.b64encode(image_bytes).decode("utf-8")
    html = f'<img src="data:image/png;base64,{decoded_img_bytes}" style="width: {width}px;" />'
    display(HTML(html))

In [ ]:
import nest_asyncio
from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod

nest_asyncio.apply()  # Required for Jupyter Notebook to run async functions

display_image(
    graph.get_graph().draw_mermaid_png(
        curve_style=CurveStyle.LINEAR,
        node_colors=NodeColors(start="#ffdfba", end="#baffc9", other="#fad7de"),
        wrap_label_n_words=9,
        output_file_path=None,
        draw_method=MermaidDrawMethod.PYPPETEER,
        background_color="white",
        padding=10,
    )
)

In [ ]:
config = {
    "configurable": {
        "thread_id": "bla",
        "wallet_id": "adasdasdad",
        "chat_id": "bla"
    }
}


In [ ]:
async for s in graph.astream(
        {
            "messages": [
                "Hi, how are you?"
            ],
        },
        config,
        stream_mode="values"
):
    if "__end__" not in s:
        s["messages"][-1].pretty_print()

In [ ]:
(await graph.aget_state(config)).values.get("used_agents", [])

In [ ]:
import pandas as pd
import aiohttp
from datetime import datetime, timedelta

In [ ]:
async def fetch_data(symbol, interval, days=None, limit=1):
    binance_interval = interval
    if days is not None:
        start_time = int((datetime.utcnow() - timedelta(days=days)).timestamp() * 1000)
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={binance_interval}&startTime={start_time}"
    else:
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={binance_interval}&limit={limit}"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            data = await resp.json()
    
    # Convert data to DataFrame
    columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
               'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
               'Taker buy quote asset volume', 'Ignore']
    df = pd.DataFrame(data, columns=columns)
    df['Date'] = pd.to_datetime(df['Open time'], unit='ms')
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    return df


In [ ]:
df = await fetch_data('BTCUSDT', "30m", )

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sample function to calculate support and resistance levels
def find_levels(df, timeframe, min_touches=2, tolerance=0.001):
    """
    Finds support and resistance levels from a higher timeframe.
    
    Args:
        df (pd.DataFrame): DataFrame containing OHLCV data.
        timeframe (str): The timeframe to identify levels (e.g., '4H', 'D').
        min_touches (int): Minimum number of touches required to validate a level.
        tolerance (float): Tolerance for levels to consider them touched multiple times.
        
    Returns:
        list: List containing resistance and support levels.
    """
    df_resampled = df.resample(timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    
    levels = []
    for i in range(2, len(df_resampled) - 2):
        if is_support(df_resampled, i):
            levels.append(df_resampled['low'][i])
        elif is_resistance(df_resampled, i):
            levels.append(df_resampled['high'][i])
    
    # Filter levels by minimum touches
    filtered_levels = []
    for level in levels:
        count = sum([abs(level - l) < tolerance for l in levels])
        if count >= min_touches and level not in filtered_levels:
            filtered_levels.append(level)
    
    return filtered_levels

def is_support(df, i):
    return df['low'][i] < df['low'][i - 1] and df['low'][i] < df['low'][i + 1] and df['low'][i + 1] < df['low'][i + 2] and df['low'][i - 1] < df['low'][i - 2]

def is_resistance(df, i):
    return df['high'][i] > df['high'][i - 1] and df['high'][i] > df['high'][i + 1] and df['high'][i + 1] > df['high'][i + 2] and df['high'][i - 1] > df['high'][i - 2]

# Plotting function to visualize support and resistance levels
def plot_levels(df, levels):
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['close'], label='Close Price', color='blue')
    for level in levels:
        plt.axhline(y=level, color='r', linestyle='--', alpha=0.5)
    plt.title('Support and Resistance Levels')
    plt.legend()
    plt.show()


In [ ]:
df = await fetch_data('4h')
# Parse dates and set index for df dataframe
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


In [ ]:
df

In [ ]:
levels = find_levels(df, '1h')

In [ ]:
levels 

In [ ]:
plot_levels(df, levels)

In [ ]:

async def get_top_200_coins():
    url = "https://api.binance.com/api/v3/ticker/24hr"
    
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            data = await resp.json()
    
    # Sort coins by quote volume to get the top 100
    sorted_data = sorted(data, key=lambda x: float(x['quoteVolume']), reverse=True)
    top_100_symbols = [item['symbol'] for item in sorted_data if item['symbol'].endswith('USDT')][:200]
    return top_100_symbols

In [ ]:
top_200_coins =await get_top_200_coins()

In [ ]:
filtered_coins = ['BTC', 'ETH'] + [coin[:-4] for coin in top_200_coins if not any(substring in coin[:-4] for substring in ['BTC', 'USD', 'ETH'])]
filtered_coins
